In [4]:
import simpy
import random
import statistics

# =====================================================
# 0. RANDOM SEED (FOR FIXED OUTPUT)
# =====================================================
random.seed(42)

# =====================================================
# 1. SYSTEM PARAMETERS
# =====================================================

SIM_TIME = 480                 # 8:00 AM – 4:00 PM (minutes)
REPLICATIONS = 10

# Clinic dispensary configuration
COUNTERS = 3
PHARMACISTS_PER_COUNTER = 2
TOTAL_PHARMACISTS = COUNTERS * PHARMACISTS_PER_COUNTER

# Empirical inter-arrival times (minutes)
EMPIRICAL_INTERARRIVALS = [
    0.8, 0.8, 1.0, 1.2, 0.9, 0.7,
    1.1, 0.5, 0.6, 1.0, 0.8, 0.9
]

# Empirical service times (minutes)
SERVICE_TIMES = [5, 6, 7]

# Additional barcode / coordination delay
BARCODE_DELAY = 1


# =====================================================
# 2. METRICS CLASS
# =====================================================

class Metrics:
    def __init__(self):
        self.waiting_times = []
        self.queue_lengths = []
        self.busy_time = 0.0
        self.completed = 0


# =====================================================
# 3. PATIENT PROCESS
# =====================================================

def patient(env, pharmacists, metrics, fast_service=False):
    arrival_time = env.now
    metrics.queue_lengths.append(len(pharmacists.queue))

    with pharmacists.request() as req:
        yield req

        wait = env.now - arrival_time
        metrics.waiting_times.append(wait)

        # Barcode delay
        yield env.timeout(BARCODE_DELAY)

        # Service time
        if fast_service:
            service_time = random.uniform(4, 5)
        else:
            service_time = random.choice(SERVICE_TIMES)

        start = env.now
        yield env.timeout(service_time)

        metrics.busy_time += env.now - start
        metrics.completed += 1


# =====================================================
# 4. ARRIVAL PROCESS
# =====================================================

def arrival_generator(env, pharmacists, metrics, interarrival_times, fast_service):
    while True:
        env.process(patient(env, pharmacists, metrics, fast_service))
        yield env.timeout(random.choice(interarrival_times))
        if env.now > SIM_TIME:
            break


# =====================================================
# 5. SINGLE SIMULATION RUN
# =====================================================

def run_simulation(interarrival_times, num_pharmacists, fast_service=False):
    env = simpy.Environment()
    pharmacists = simpy.Resource(env, capacity=num_pharmacists)
    metrics = Metrics()

    env.process(arrival_generator(
        env, pharmacists, metrics, interarrival_times, fast_service
    ))

    env.run(until=SIM_TIME)

    utilization = metrics.busy_time / (num_pharmacists * SIM_TIME)

    return {
        "Avg Wait (min)": statistics.mean(metrics.waiting_times),
        "Avg Queue": statistics.mean(metrics.queue_lengths),
        "Throughput/hr": (metrics.completed / SIM_TIME) * 60,
        "Utilization (%)": utilization * 100,
        "Patients Served": metrics.completed
    }


# =====================================================
# 6. MULTIPLE REPLICATIONS
# =====================================================

def experiment(interarrival_times, num_pharmacists, fast_service=False):
    results = [
        run_simulation(interarrival_times, num_pharmacists, fast_service)
        for _ in range(REPLICATIONS)
    ]

    return {
        "Avg Wait (min)": statistics.mean(r["Avg Wait (min)"] for r in results),
        "Avg Queue": statistics.mean(r["Avg Queue"] for r in results),
        "Throughput/hr": statistics.mean(r["Throughput/hr"] for r in results),
        "Utilization (%)": statistics.mean(r["Utilization (%)"] for r in results),
        "Patients Served": statistics.mean(r["Patients Served"] for r in results)
    }


# =====================================================
# 7. SCENARIOS
# =====================================================

results = []

# Scenario 1: Baseline
results.append({
    "Scenario": "Baseline (3 Counters, 6 Pharmacists)",
    **experiment(EMPIRICAL_INTERARRIVALS, TOTAL_PHARMACISTS)
})

# Scenario 2: +1 Counter
results.append({
    "Scenario": "+1 Counter (4 Counters, 8 Pharmacists)",
    **experiment(EMPIRICAL_INTERARRIVALS, 8)
})

# Scenario 3: Faster Service
results.append({
    "Scenario": "Faster Service (4–5 min)",
    **experiment(EMPIRICAL_INTERARRIVALS, TOTAL_PHARMACISTS, fast_service=True)
})


# =====================================================
# 8. FORMATTED OUTPUT (LIKE IMAGE STYLE)
# =====================================================

print("=" * 60)
print("HORANA DISTRICT GENERAL HOSPITAL – CLINIC DISPENSARY")
print("PERFORMANCE SIMULATION")
print("Simulation Duration: 8 Hours (480 Minutes)")
print("=" * 60)

for r in results:
    print("\n" + "=" * 60)
    print(f"SCENARIO: {r['Scenario'].upper()}")
    print("=" * 60)

    print(f"Mean Waiting Time   : {r['Avg Wait (min)']:.2f} minutes")
    print(f"Mean Queue Length   : {r['Avg Queue']:.2f} patients")
    print(f"Patients Served    : {r['Patients Served']:.0f} patients")
    print(f"Throughput         : {r['Throughput/hr']:.2f} patients/hour")
    print(f"Utilization        : {r['Utilization (%)']:.2f} %")


HORANA DISTRICT GENERAL HOSPITAL – CLINIC DISPENSARY
PERFORMANCE SIMULATION
Simulation Duration: 8 Hours (480 Minutes)

SCENARIO: BASELINE (3 COUNTERS, 6 PHARMACISTS)
Mean Waiting Time   : 63.08 minutes
Mean Queue Length   : 73.51 patients
Patients Served    : 406 patients
Throughput         : 50.80 patients/hour
Utilization        : 84.76 %

SCENARIO: +1 COUNTER (4 COUNTERS, 8 PHARMACISTS)
Mean Waiting Time   : 4.98 minutes
Mean Queue Length   : 5.35 patients
Patients Served    : 540 patients
Throughput         : 67.51 patients/hour
Utilization        : 84.23 %

SCENARIO: FASTER SERVICE (4–5 MIN)
Mean Waiting Time   : 14.40 minutes
Mean Queue Length   : 16.25 patients
Patients Served    : 518 patients
Throughput         : 64.80 patients/hour
Utilization        : 80.95 %
